# Separating orthoimages into the individual manual labels

In [ ]:
# Import necessary packages
import os
import time
import cv2
import numpy as np
from PIL import Image
from skimage.measure import label

### Functions

In [ ]:
def bbox2(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]

    return rmin, rmax, cmin, cmax

def squared_image(img_1):
    # check what is the biggest dimension
    max = np.max(img_1.shape)
    # create a blank image with the maximum size
    img_pad_1 = np.zeros((max, max, 3),dtype='uint8' )

    # check which dimension is the biggest
    dim_max = np.argmax(img_1.shape)
    # add the image in the other dimension
    if dim_max == 0:
        difference = int((max-img_1.shape[1])/2)
        img_pad_1[:,difference :difference+img_1.shape[1], : ] = img_1

    elif dim_max == 1:
        difference = int((max-img_1.shape[0])/2)
        img_pad_1[difference :difference+img_1.shape[0], : ] = img_1
    return img_pad_1

## Import the orthoimage and the corresponding manual labels as tif files

In [ ]:
tif_file_path = "insert orthoimage tif file path"
segmentation_file_path = "insert manual segmentation tif file path"

## Add the output folder path, and the identifier for the files

In [ ]:
folder_path=" insert output folder path"
fn='insert name here'

## Create the individual labels in the specified folder

In [ ]:
with Image.open(segmentation_file_path) as im:
    image_labels=np.array(im, dtype="uint8")
    image_labels[image_labels == 128] = 0
    with Image.open(tif_file_path) as img:
        img_g=cv2.cvtColor(np.array(img, dtype='uint8'), cv2. COLOR_RGBA2RGB)
        img_separate_labels=label(image_labels)
        mask = np.zeros(img_g.shape, dtype='uint8')
        for segVal in np.unique(img_separate_labels)[1:]:
            mask = mask*0
            mask[img_separate_labels == segVal] = ~0
            label_mask=np.bitwise_and(mask[:,:,0], image_labels)
            new_mask=np.bitwise_and(mask, img_g)
            y1, y2, x1, x2=bbox2(new_mask[:,:,0])
            new_image=new_mask[y1:y2, x1:x2, : ]
            filename=fn+ time.strftime("%Y%m%d%H%M%S")+'class'+str(np.max(label_mask) )+'.png'
            new_image2=squared_image(new_image)
            cv2.imwrite(os.path.join(folder_path , filename), cv2.cvtColor(new_image2, cv2.COLOR_RGB2BGR))
            cv2.waitKey(0)